In [1]:
# Inspired by https://www.sciencedirect.com/science/article/pii/S0377221723000735
# https://www.sciencedirect.com/science/article/pii/S0167637704000537
# https://medium.com/swlh/techniques-for-subtour-elimination-in-traveling-salesman-problem-theory-and-implementation-in-71942e0baf0c

In [2]:
import gurobipy as gp
import math
import pandas as pd

In [3]:
# Read data

# Read files
file = "Information Scrapyard challenge.xlsx"
production_plan = pd.read_excel(file, sheet_name = "Production plan")

grade_mix = pd.read_excel(file, sheet_name = "Grade mix large buckets")
grade_mix.columns.values[0] = "Grade"

scrap_types = pd.read_excel(file, sheet_name = "Scraptype")
scrap_types.columns.values[0] = "Scraptype"
scrap_types.columns.values[9] = "Capacity"
# print(f"{scrap_types}")

# Store density order and capacity of each scrap type
scrap_density = {}
scrap_storage = {}
for index, row in scrap_types.iloc[0:32,:].iterrows():
    scrap_density[row["Scraptype"]] = row["Layer bucket"]
    scrap_storage[row["Scraptype"]] = row["Capacity"]

set_all_scraps = range(0, 32)

#print(f"{scrap_density}")
#print(f"{scrap_storage}")

# Create a set of all grades produced and a list of the quantities produced
all_grades_produced = production_plan.query("Installation=='EAF1' | Installation=='EAF2'")["Grade"]
grades_produced = list(set(all_grades_produced))
grades_count_dict = all_grades_produced.value_counts().to_dict()
all_grades_produced = [grade.removesuffix("_EAF") for grade in all_grades_produced]
grades_produced = [grade.removesuffix("_EAF") for grade in grades_produced]

# Create a dictionary which holds the recipe for each grade
grade_recipe_dict = {}
for grade in grades_produced:
    row = grade_mix.loc[grade_mix["Grade"] == grade].iloc[:,2:].values.tolist()[0]
    grade_recipe_dict[grade] = row

# List all unused scrap types
scrap_types_unused = list(range(0, 32))
for row in grade_recipe_dict.values():
    for i, value in enumerate(row):
        if value > 0 and i in scrap_types_unused:
            scrap_types_unused.remove(i)
print(f"Unused scrap types :{scrap_types_unused}")

# Create dict to map new scrap indices to old scrap indices
new_to_old = {}
new_index = 0
for old_scrap in set_all_scraps:
    if old_scrap in scrap_types_unused:
        continue
    new_to_old[new_index] = old_scrap
    new_index += 1
#print(f"{new_to_old}")

# Remove unused scrap types from rows in dictionary
for key, value in grade_recipe_dict.items():
    new = [value[i] for i in set_all_scraps if i not in scrap_types_unused]
    grade_recipe_dict[key] = new
print(f"{grade_recipe_dict}")

# Transform the recipes into a list of scrap types used
recipes = []
recipes_quantities = []
for r_i, tons in grade_recipe_dict.items():
    recipe = []
    for i, value in enumerate(tons):
        if value > 0:
            recipe.append(i)
    recipes.append(recipe)
    recipes_quantities.append(grades_count_dict[r_i + "_EAF"])
print(f"{recipes_quantities}")

# Store the scrap types that must precede each scrap type in a recipe
recipes_preceders = []
for recipe in recipes:
    preceders = {}
    for scrap in recipe:
        preceders[scrap] = [s for s in recipe if scrap_density[new_to_old[s]] < scrap_density[new_to_old[scrap]]]
    recipes_preceders.append(preceders)

#recipes_preceders[0]

scrap_capacity = [scrap_storage[s] for s in range(0, 32) if s not in scrap_types_unused]

# Set number of scraps used
scraps = 32 - len(scrap_types_unused)

# Set maximum number of boxes
max_boxes = 39
max_points = max_boxes * 2 + 2

yard_length = 505
min_box_length = 10
wall_length = 1.7
box_width = 40
box_height = 10

set_scraps = range(0, scraps)
set_boxes = range(0, max_boxes)
set_points = range(0, max_points)
set_recipes = range(0, len(recipes))

Unused scrap types :[2, 10, 11, 26, 30, 31]
{'K570': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 48.40000000000001, 12.0, 0.0, 76.4, 0.0, 0.0, 0.0, 1.5, 0.0, 0.0, 0.0, 0.0, 10.0, 12.0, 0.0, 0.0, 5.0], 'KT70': [0.0, 0.0, 0.0, 12.0, 0.0, 14.0, 7.0, 64.0, 19.0, 0.0, 0.0, 0.0, 29.4, 31.4, 7.0, 3.0, 0.0, 21.3, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 3.0, 0.0], 'L640': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 66.4, 16.0, 8.0, 7.800000000000001, 21.7, 5.0, 0.0, 0.9, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 0.0, 2.0, 5.0], 'LK60': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 17.5, 0.0, 0.0, 0.0, 0.0, 0.0, 97.8, 0.0, 0.0, 10.0, 0.0, 0.0, 0.0, 5.0], 'L650': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 88.8, 0.0, 0.0, 41.9, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 0.0, 0.0, 0.0, 0.0], 'KM80': [5.4, 0.0, 2.0, 0.0, 4.0, 5.0, 0.0, 22.5, 5.0, 0.0, 0.0, 0.0, 0.0, 27.9, 6.0, 0.0, 0.0, 150.3, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0], 'T280': [27.5, 0.0, 13.0, 3.0, 0.0, 7.0, 2.0, 17.0, 5.0, 0.0,

In [4]:
# Override parameters for testing
scraps = 6 #32
max_boxes = 6 #39
max_points = max_boxes * 2 + 2

yard_length = 505
min_box_length = 10
wall_length = 1.7
box_width = 20
box_height = 10

set_scraps = range(0, scraps)
set_boxes = range(0, max_boxes)
set_points = range(0, max_points)

scrap_capacity = [4000, 2000, 2000, 2000, 2000, 2000]

# Minium of 3 scraps per recipe, MUST USE ALL SCRAPS IN AT LEAST ONE RECIPE
recipes = [[0, 2, 1, 3], [1, 2, 3], [3, 2, 4, 5], [4, 3, 2]]
recipes_quantities = [1, 1, 1, 1]
recipes_strict_order_i = [[0, 3], [0, 1, 2], [0, 1, 2], [0, 1, 2]]
recipes_preceders = []
for r_i, recipe in enumerate(recipes):
    preceders_dict = {}
    for s_i, scrap in enumerate(recipe):
        if s_i in recipes_strict_order_i[r_i]:
            preceders_dict[scrap] = recipe[:s_i]
        else:
            preceders_dict[scrap] = [recipe[i] for i in range(0, len(recipe)) if i in recipes_strict_order_i[r_i] and i < s_i]
    recipes_preceders.append(preceders_dict)
       
set_recipes = range(0, len(recipes))
recipes_preceders

[{0: [], 2: [0], 1: [0], 3: [0, 2, 1]},
 {1: [], 2: [1], 3: [1, 2]},
 {3: [], 2: [3], 4: [3, 2], 5: [3, 2, 4]},
 {4: [], 3: [4], 2: [4, 3]}]

In [5]:
# Create a new model
m = gp.Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-02


In [6]:
# Create variables

# Scrap s is assigned to box b
x = {}
for scrap in set_scraps:
    for box in set_boxes:
        x[scrap, box] = m.addVar(vtype='B', name=f"x[{scrap},{box}]")

# Scrap s is assigned to point w
a = {}
for scrap in set_scraps:
    for point in set_points[1:]:
        a[scrap, point] = m.addVar(vtype='B', name=f"a[{scrap},{point}]")

# Box b is in solution path for recipe r
y = {}
for box in set_boxes:
    for recipe in set_recipes:
        y[box, recipe] = m.addVar(vtype='B', name=f"y[{box},{recipe}]")

# Box b is in the global solution
g = {}
for box in set_boxes:
    g[box] = m.addVar(vtype='B', name=f"g[{box}]")

# Distance between start and point w
w = {}
for w1 in set_points:
    w[w1] = m.addVar(vtype='C', ub=yard_length, name=f"w[{w1}]")

# Distance between two points w1 and w2
d = {}
for w1 in set_points:
    for w2 in set_points:
        if w1 == w2: continue
        d[w1, w2] = m.addVar(vtype='C', ub=yard_length, name=f"d[{w1},{w2}]")

# Length of box b
l = {}
for box in set_boxes:
    l[box] = m.addVar(vtype='C', ub=yard_length, name=f"l[{box}]")

# Point w is in solution path for recipe r
p = {}
for w1 in set_points:
    for recipe in set_recipes:
        p[w1, recipe] = m.addVar(vtype='B', name=f"p[{w1},{recipe}]")

# Edge w1 -> w2 is in solution path for recipe r
z = {}
for w1 in set_points:
    for w2 in set_points:
        if w1 != w2:
            for recipe in set_recipes:
                z[w1, w2, recipe] = m.addVar(vtype='B', name=f"z[{w1},{w2},{recipe}]")

# Capacity of box b
c = {}
for box in set_boxes:
    c[box] = m.addVar(vtype='C', name=f"c[{box}]")

# MTZ value in a point in a recipe
mtz = {}
for recipe in set_recipes:
    for w1 in set_points[1:]:
        mtz[w1, recipe] = m.addVar(vtype='C', name=f"mtz[{w1},{recipe}]")

# Actual MTZ for scraps in a recipe
mtz_s = {}
for recipe in set_recipes:
    for scrap in set_scraps:
        mtz_s[scrap, recipe] = m.addVar(vtype='C', name=f"mtz_s[{scrap},{recipe}]")

# MTZ for a scrap in a point
mtz_s_a = {}
for recipe in set_recipes:
    for scrap in set_scraps:
        for point in set_points[1:]:
            mtz_s_a[scrap, point, recipe] = m.addVar(vtype='C', name=f"mtz_s_a[{scrap},{point},{recipe}]")

# Distance of path of recipe r (for use in objective function)
r_d = {}
for recipe in set_recipes:
    r_d[recipe] = m.addVar(vtype='C', name=f"r_d[{recipe}]")


In [7]:
# Add constraints

# Each scrap used in a recipe is assigned to at least one box
for s in set_scraps:
    m.addConstr(sum(x[s, b] for b in set_boxes) >= 1)

# point 0 is in solution path for all recipes
for r in set_recipes:
    m.addConstr(p[0, r] == 1)

# Each box is assigned at most one scrap
for b in set_boxes:
    m.addConstr(sum(x[s, b] for s in set_scraps) <= g[b])

# The distance to point 1 is 100m
m.addConstr(w[1] == 100)
m.addConstr(w[0] == 0)

# Dummy points are not in the solution path for any recipe
for r in set_recipes:
    m.addConstr(p[max_points - 1, r] == 0)

# Capacity of a box is determined by the formula
for b in set_boxes:
    m.addConstr(c[b] == box_height * box_width * l[b])

# The total capacity of boxes with a scrap s must be greater than the required volume
for s in set_scraps:
    m.addConstr(sum(c[b] * x[s, b] for b in set_boxes) >= scrap_capacity[s])

# Points p1 and p2 of box b are assigned to the same scrap
for b in set_boxes:
    for s in set_scraps:
        m.addConstr(x[s, b] == a[s, 2*b + 1])
        m.addConstr(x[s, b] == a[s, 2*b + 2])

# The number of boxes in a solution for recipe r is equal to the number of scraps required by the recipe
for r in set_recipes:
    m.addConstr(sum(y[b, r] for b in set_boxes) == len(recipes[r]))

# There is exactly one box b in the solution path for recipe r that corresponds to each scrap s
for r in set_recipes:
    for s in recipes[r]:
        m.addConstr(sum(x[s, b] * y[b, r] for b in set_boxes) == 1)

# There is one incoming edge to each point w in the solution path for recipe r
for r in set_recipes:
    for w2 in set_points:
        m.addConstr(sum(z[w1, w2, r] for w1 in set_points if w1 != w2) == p[w2, r])

# There is one outgoing edge from each point w in the solution path for recipe r
for r in set_recipes:
    for w1 in set_points:
        m.addConstr(sum(z[w1, w2, r] for w2 in set_points  if w1 != w2) == p[w1, r])

# The length of a box is the distance between the first and last point of the box
for b in set_boxes:
    m.addConstr(l[b] == w[2*b + 2] - w[2*b + 1])

# The length of a box in the solution is larger than the minimum
for b in set_boxes:
    m.addGenConstrIndicator(g[b], True, l[b] >= min_box_length)

# The length of a box not in the solution is 0
for b in set_boxes:
    m.addGenConstrIndicator(g[b], False, l[b] == 0)

# The distance between edge points of a wall (in the solution) is equal to the length of the wall
for b in set_boxes:
    m.addConstr(w[2*b + 3] - w[2*b + 2] == wall_length * g[b])
    # m.addGenConstrIndicator(g[b], True, w[2*b + 3] - w[2*b + 2] == 1.7)

# Calculate the distances between all points
for w1 in set_points:
    for w2 in set_points:
        if w2 < w1 or w1 == w2: continue
        m.addConstr(d[w1, w2] == w[w2] - w[w1])
        m.addConstr(d[w2, w1] == d[w1, w2])

# Box b is in the global solution if it is in the solution path for any recipe
for b in set_boxes:
    m.addGenConstrIndicator(g[b], True, sum(y[b, r] for r in set_recipes) >= 1)

# Box b is not in the solution path for any recipe if it is not in the global solution
for b in set_boxes:
    for r in set_recipes:
        m.addConstr(y[b, r] <= g[b])

# If a box b is not in the global solution, all later boxes are also not in the global solution
for b1 in set_boxes:
    m.addConstr(sum(g[b2] for b2 in set_boxes if b2 > b1) <= g[b1] * max_boxes)

# If a box is in the solution path for recipe r, then one of the edge points of the box is also in the solution path for recipe r
for r in set_recipes:
    for b in set_boxes:
        m.addConstr(y[b, r] == p[2*b + 1, r] + p[2*b + 2, r])

# Only one edge of a box can be in the solution path for recipe r
for r in set_recipes:
    for b in set_boxes:
        m.addConstr(p[2*b + 1, r] + p[2*b + 2, r] <= 1)

# The distance of a recipe is the sum of the distances between the points in the solution path
for r in set_recipes:
    m.addConstr(r_d[r] == sum(z[w1, w2, r] * d[w1, w2] for w1 in set_points for w2 in set_points if w1 != w2))

# Ordering constraints
# Ordering constraints WORKS WELL ONLY FOR STRICT ORDERING
# for r in set_recipes:
#     for w1 in set_points[1:]:
#         for w2 in set_points[1:]:
#             if w1 == w2: continue
#             m.addConstr(z[w1, w2, r] <= sum(a[recipes[r][i], w1] * a[recipes[r][i+1], w2] for i in range(len(recipes[r])-1)))
            
# No subtours in recipe solution paths using MTZ values
for r in set_recipes:
    for w1 in set_points[1:]:
        for w2 in set_points[1:]:
            if w1 == w2: continue
            # m.addConstr(mtz[w2, r] >= mtz[w1, r] + 1 - 1000 * (1 - z[w1, w2, r]))
            m.addGenConstrIndicator(z[w1, w2, r], True, mtz[w2, r] >= mtz[w1, r] + 1)

# MTZ values for points not in the solution path are 0
for r in set_recipes:
    for w1 in set_points[1:]:
        m.addGenConstrIndicator(p[w1, r], False, mtz[w1, r] == 0)

# MTZ values for scrap ordering are equal to the MTZ value of a point if the scrap is assigned to the point
for r in set_recipes:
    for s in set_scraps:
        for w1 in set_points[1:]:
            m.addConstr(mtz_s_a[s, w1, r] == mtz[w1, r] * a[s, w1])

# MTZ values for scrap ordering are equal to the maximum mtz value of the points assigned to the scrap
for r in set_recipes:
    for s in set_scraps:
        m.addGenConstrMax(mtz_s[s, r], [mtz_s_a[s, w1, r] for w1 in set_points[1:]], name=f"max_mtz_s[{s},{r}]")

# Precendence of scrap types in recipes
for r in set_recipes:
    for s_recipe in recipes[r]:
        for s_precedence in recipes_preceders[r][s_recipe]:
            m.addConstr(mtz_s[s_recipe, r] >= mtz_s[s_precedence, r] + 1)



In [8]:
# Set objective function

# Travel distance
# objective = gp.quicksum(d[w1, w2] * z[w1, w2, r] for w1 in set_points for w2 in set_points for r in set_recipes if w1 != w2)
objective = gp.quicksum(r_d[r] * recipes_quantities[r] for r in set_recipes)

# Length of yard
# objective = gp.quicksum(l[b] for b in set_boxes)

m.setObjective(objective, gp.GRB.MINIMIZE)

In [9]:
# Statistics
m.printStats()


Statistics for model Unnamed:
  Linear constraint matrix    : 0 Constrs, 0 Vars, 0 NZs
  Matrix coefficient range    : [ 0, 0 ]
  Objective coefficient range : [ 0, 0 ]
  Variable bound range        : [ 0, 0 ]
  RHS coefficient range       : [ 0, 0 ]


In [10]:
# Set initial solution
lengths_l = [10.0,69.4,29.8,16.6,10.0,10.0,10,12.2,22.4,13.6,14.9,26.8,37.8,30.4,23.2,26.8,]
lengths_r = [15.4,24.8,26.2,35.9,10.0,11.3,10.0,10.0,11.3,10.0,10.0,40.4,53.8,14.9,51.1,30.4]
scrap_type_l = [10,18,8,28,4,11,1,9,22,30,17,16,21,24,19,27]
scrap_type_r = [26,13,6,12,5,2,7,23,0,3,29,15,14,31,20,25]

unfiltered_lengths = lengths_l + lengths_r
unfiltered_scrap_types = scrap_type_l + scrap_type_r
filtered_scrap_types = []
filtered_lengths = []

for i, scrap_type in enumerate(unfiltered_scrap_types):
    if scrap_type in scrap_types_unused:
        continue
    else:
        filtered_scrap_types.append(scrap_type)
        filtered_lengths.append(unfiltered_lengths[i])

start_scrap_types = []
for scrap in filtered_scrap_types:
    decrement = 0
    for unused in scrap_types_unused:
        if scrap > unused:
            decrement -= 1
    start_scrap_types.append(scrap + decrement)

# for i, length in enumerate(filtered_lengths):
#     l[i].start = length
#     g[i].start = 1
#     x[start_scrap_types[i], i].start = 1

for i in set_scraps:
    l[i].start = 55
    g[i].start = 1
    x[i, i].start = 1


In [10]:
# Solve
m.Params.timeLimit = 60 * 90
m.params.cuts = 0
m.params.cutpasses = 0
# Focus on finding feasable solutions
m.params.mipfocus = 1
m.optimize()

Set parameter TimeLimit to value 5400
Set parameter Cuts to value 0
Set parameter CutPasses to value 0
Set parameter MIPFocus to value 1
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 9 PRO 7940HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 505 rows, 1528 columns and 2550 nonzeros
Model fingerprint: 0x0e6af53d
Model has 336 quadratic constraints
Model has 718 general constraints
Variable types: 600 continuous, 928 integer (928 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 5e+02]
  RHS range        [1e+00, 1e+02]
  QRHS range       [1e+00, 4e+03]
  GenCon rhs range [1e+00, 1e+01]
  GenCon coe range [1e+00, 1e+00]

User MIP start produced solution with objective 2724

: 

In [11]:
# Store solution
if m.SolCount > 0:
    m.write("solution.sol")

In [12]:
# # Read solution
# m.update()
# m.read("solution.sol")
# m.Params.timeLimit = 2
# m.optimize()

In [13]:
# Show solution
print(f"Objective value: {m.objVal}")

# Print boxes in solution
print(f"Boxes in solution: {[box for box in (set_boxes) if g[box].X > 0]}")

# Print box capacities
for box in set_boxes:
    if c[box].X > 0:
        print(f"Box {box} capacity: {c[box].X}")

# Print scrap assignments
scrap_assignments = {}
for box in set_boxes:
    for scrap in set_scraps:
        if x[scrap, box].X > 0:
            scrap_assignments[box] = scrap
            print(f"Scrap {scrap} is assigned to box {box}")

# Print box lengths
for box in set_boxes:
    if l[box].X > 0:
        print(f"Box {box} length: {l[box].X}")

# Print part of point distances
for w1 in set_points[:-1]:
    w2 = w1 + 1
    if d[w1, w2].X > 0:
        print(f"Distance between point {w1} and point {w2}: {d[w1, w2].X}")

# Print solution paths
def get_box_index(point):
    return math.floor((point - 1) // 2)

# for w1 in set_points[1:]:
#     for w2 in set_points[1:]:
#         if w1 == w2: continue
#         if z[w1, w2, 0].X > 0:
#             print(f"MTZ {mtz[w1, 0].X} Box {get_box_index(w1)} -> MTZ {mtz[w2, 0].X} Box {get_box_index(w2)}")

for recipe in set_recipes:
    print(f"Recipe {recipe} scrap mtz:")
    for scrap in set_scraps:
        print(f"MTZ scrap {scrap} {mtz_s[scrap, recipe].X}")

for recipe in set_recipes:
    print(f"Recipe {recipe} solution path:")
    for w1 in set_points:
        for w2 in set_points:
            if w1 != w2 and z[w1, w2, recipe].X > 0:
                # print(f" {w1} -> {w2}")
                if w1 == 0:
                    print(f"Start -> Box {get_box_index(w2)}: Scrap: {scrap_assignments[get_box_index(w2)]}")
                elif w2 == 0:
                    print(f"Box {get_box_index(w1)}: Scrap: {scrap_assignments[get_box_index(w1)]} -> Start")
                else:
                    print(f"Box {get_box_index(w1)}: Scrap: {scrap_assignments[get_box_index(w1)]} -> Box {get_box_index(w2)}: Scrap: {scrap_assignments[get_box_index(w2)]}")
    

Objective value: 69.99999999999011
Boxes in solution: [0, 1, 2, 3, 4, 5]
Box 0 capacity: 2000.0
Box 1 capacity: 4000.0
Box 2 capacity: 2000.0
Box 3 capacity: 2000.0
Box 4 capacity: 2000.0
Box 5 capacity: 2000.0
Scrap 5 is assigned to box 0
Scrap 0 is assigned to box 1
Scrap 1 is assigned to box 2
Scrap 4 is assigned to box 3
Scrap 3 is assigned to box 4
Scrap 2 is assigned to box 5
Box 0 length: 10.0
Box 1 length: 20.0
Box 2 length: 10.0
Box 3 length: 10.0
Box 4 length: 10.0
Box 5 length: 10.0
Distance between point 0 and point 1: 100.0
Distance between point 1 and point 2: 10.0
Distance between point 2 and point 3: 1.7
Distance between point 3 and point 4: 20.0
Distance between point 4 and point 5: 1.7
Distance between point 5 and point 6: 10.0
Distance between point 6 and point 7: 1.7
Distance between point 7 and point 8: 10.0
Distance between point 8 and point 9: 1.7
Distance between point 9 and point 10: 10.0
Distance between point 10 and point 11: 1.7
Distance between point 11 and

KeyError: 6